# First sample about LLama-Cpp-Python (Local model deployment)

## Installation
1. Fork https://github.com/abetlen/llama-cpp-python/forkand clone       
```
git clone https://github.com/<your-git-id>/llama-cpp-python llama
```
2. Update the source tree (download llama-cpp)
```
cd llama
git pull origin
git submodule init
git submodule update
```
3. Install the llama-cpp-python
```
python -m pip install --upgrade --force-reinstall --no-cache-dir .
```
If encountering pip SSL error, try this
```
python3 -m pip install "ipykernel>=6.9.2" -U --user --force-reinstall --trusted-host pypi.org --trusted-host files.pythonhosted.or
```

## Installation and model loading

Update Jupyter Notebook if neccessary.

In [ ]:
!pip3 install -U jupyter                                                                                                  

Install prequisitive modules.

In [ ]:
!pip3 install huggingface_hub

In [ ]:
!pip3 uninstall wasabi -y
!pip3 install wasabi==0.9.1 

In [3]:
# Fix the SSL wrong version number bug in OSX python 3.9 urllib
!python3 -m pip install --upgrade urllib3

In [4]:
import llama_cpp

llama = llama_cpp.Llama.from_pretrained(
    #repo_id="TheBloke/Llama-2-7B-GGUF",
    #filename="llama-2-7b.Q8_0.gguf",
    repo_id="TheBloke/Llama-2-7B-Chat-GGML",
    filename="llama-2-7b-chat.ggmlv3.q8_0.bin",
    verbose=True
)



llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/lihw/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-GGUF/snapshots/b4e04e128f421c93a5f1e34ac4d7ca9b0af47b80/./llama-2-7b.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count 

In [5]:
llama.verbose = False

## Simple response

In [6]:
def completion(prompt, temperature = 0.2, top_p = 0.95):
    response = llama.create_completion(prompt = prompt,
                                       max_tokens=32, # Generate up to 32 tokens, set to None to generate up to the end of the context window
                                       stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
                                       echo=True # Echo the prompt back in the output
    ) 

    res = ""
    for chunk in response["choices"]:
        if "text" not in chunk:
            continue
        res += chunk["text"]

    return res

In [7]:
prompt = "Q: Name the planets in the solar system. A:"

response = completion(prompt, temperature = 0.01)

print(response)

Q: Name the planets in the solar system. A: 39, 40, or 41 mg/dl (mmol/L).


Compare to the remote deployment of Llama2. The response is almost the same.

In [ ]:
!pip3 install langchain
!pip3 install replicate

In [ ]:
import os

# Get a free API key from https://replicate.com/account/api-tokens
os.environ["REPLICATE_API_TOKEN"] = "r8_VqKW8H8qmQoCyCsRGWwEMJB0aXKvbtN49QonG"

In [ ]:
from langchain.llms import Replicate

def remote_completion(
    prompt: str,
    model: str = "meta/llama-2-7b-chat",
    temperature: float = 0.2,
    top_p: float = 0.95,
) -> str:
    print(prompt)
    llm = Replicate(
        model=model,
        model_kwargs={"temperature": temperature,"top_p": top_p, "max_new_tokens": 1000}
    )
    return llm(prompt)


In [ ]:
response = remote_completion(prompt = prompt, model = "meta/llama-2-7b-chat", temperature = 0.01)
print(response)

## Chat response

In [ ]:
def chat_completion(chat, temperature = 0.6, top_p = 0.9):
    response = llama.create_chat_completion(
        messages = chat,
        temperature= temperature,
        top_p = top_p   
    )

    return response


In [ ]:
chat =[
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ]
chat_completion(chat)

print("============ Response: ===========")
for chunk in response:
    delta = chunk["choices"][0]["delta"]
    if "content" not in delta:
        continue
    print(delta["content"], end="", flush=True)

print()

In [ ]:
    response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {
                "country": {"type": "string"},
                "capital": {"type": "string"}
            },
            "required": ["country", "capital"],
        }
    },
    stream=True
)

print("============ Response: ===========")
for chunk in response:
    delta = chunk["choices"][0]["delta"]
    if "content" not in delta:
        continue
    print(delta["content"], end="", flush=True)

print()